## Automate download of the zipped county files

In [ ]:
# for the webpage (per state) automate the download of the ZIP county files as outlined below

In [ ]:
# create txt log of zipfiles downloaded


In [ ]:
# do not download the file if already in the log

In [ ]:
# if not in the log or the file on website is newer

# 1. download
# 2. update log
# 3. unzip to SSURGO folder
# 4. delete the downloaded zipped file